In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Updated Cust close v1

Generate all the priv/public keys

In [3]:
funding_privkey_hex = "1111111111111111111111111111111100000000000000000000000000000000"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

change_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

cust_payout_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
cust_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_payout_privkey_hex)).hex()


merch_payout_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_payout_privkey_hex)).hex()

merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
RL = hashlib.sha256(bytes.fromhex(revocation_secret_hex)).digest()
revocation_lock_hex = RL.hex()

merch_cpfp_privkey_hex = "2222222222222222222222222222222277777777777777777777777777777777"
merch_cpfp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_cpfp_privkey_hex)).hex()

merch_fee_privkey_hex = "2222222222222222222222222222222266666666666666666666666666666666"
merch_fee_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_fee_privkey_hex)).hex()

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [4]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -minrelaytxfee=0")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [5]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2MuuVDMJqfh2J9iS3DtfUeU1tEfff1SNahi


Send btc to the escrow funder

In [6]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 3.0")
print(txid_1)

680b9d652406091413425d91963312383d4cfe8562db20c4932c184c035faa91


In [7]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [8]:
os.system("bitcoin-cli -regtest generate 1");

Generate raw escrow funding transaction

In [9]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_changev2.py" 
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --input_amount_btc " + "3.0"
                        + " --funding_privkey " + funding_privkey_hex
                        + " --escrow_value_btc " + "2"
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_change_value_btc " + "1"
                        + " --cust_change_pubkey " + change_pubkey_hex)
print(raw_escrow_tx)

0200000000010191aa5f034c182c93c420db6285fe4c3d38123396915d421314090624659d0b680100000017160014578dd1183845e18d42f90b1a9f3a464675ad2440ffffffff0200c2eb0b00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e00e1f50500000000160014fc7250a211deddc70ee5a2738de5f07817351cef02483045022100befd1551a11b91e82dd8c312ad0afd57125d8c92f9c9e1eb9b637bd84aad0989022007fdd43d00bd47738e7a5bf4aee91641410c7a55a29c63b6fe70f23c8328d7d30121021d5c5194a62b272f98a7d8321e4b3a03add9d61d13a15a00123888b81850cee200000000


Broadcast escrow funding transaction

In [10]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

5d89995763fa1d74af907abf01bd244104882505cef9bcd7fa77635162c345aa


In [11]:
# raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [12]:
os.system("bitcoin-cli -regtest generate 1");

# Cust-close from escrow

In [13]:
raw_cust_close_tx = subprocess.getoutput("python cust_closev2.py"
                + " --spend_from " + "escrow"
                + " --txid_str " + escrow_txid
                + " --index " + "0"
                + " --input_amount_btc " + "2"
                + " --cust_privkey " + cust_privkey_hex
                + " --merch_privkey " + merch_privkey_hex
                + " --cust_script_value_btc " + "1"
                + " --cust_payout_pubkey " + cust_payout_pubkey_hex
                + " --merch_payout_value_btc " + "1"
                + " --merch_payout_pubkey " + merch_payout_pubkey_hex
                + " --revocation_lock " + revocation_lock_hex
                + " --merch_dispute_pubkey " + merch_disp_pubkey_hex
                + " --to_self_delay " + "05cf")
print(raw_cust_close_tx)



decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_cust_close_tx)
# print(decoded)

02000000000101aa45c362516377fad7bcf9ce052588044124bd01bf7a90af741dfa635799895d0000000000ffffffff0300e1f5050000000022002067cb20e705c4eb4363194a74d2f743afc1c9ee3cd741d45e21268b16add04f8b00e1f50500000000160014d4354803d10e77eccfc3bf06c152ae694d05d3810000000000000000436a41f8345a21a55dc665b65c8dcfb49488b8e4f337d5c9bb843603f7222a892ce94103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd7686410400483045022100db2a7932c1a1d172ab4317be2e48d62d7c3d63fbeb1f8ddb7815e3b7129ad9210220318b76569e69068c99cea66fa1350b2249bae9b32d8717489efb158ae8c5c0b401483045022100b14d7aec705c8a22afab74a93ad3b08a555fe05e19e710ed6c5823c0de4b954002203431bbd7d87367ae1dc85db805e2b00c91e12c392f96dd772d09b91a3ec353570147522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


In [14]:
cust_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_close_tx + " true")
print(cust_close_txid)

fd6593d9cfc6cb77086c4119c8dcb9de1c2abea4468887bd874a8d1937d7c09b


In [15]:
os.system("bitcoin-cli -regtest generate 1");

In [16]:
cust_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_close_tx)
print(decoded)